In [ ]:
# Import libraries:
import os
import time
from time import sleep
import logging
import requests
import threading
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from datetime import datetime, timedelta

In [ ]:
load_dotenv()

In [ ]:
# Start logger:
logging.basicConfig(
	filename='errlog.log',
	level=logging.WARNING,
	format='%(asctime)s:%(levelname)s:%(message)s'
)

In [ ]:
# Retrieve Alpaca API keys:
alpaca_api_key=os.getenv('ALPACA_API_KEY')
alpaca_secret_key=os.getenv('ALPACA_SECRET_KEY')

In [ ]:
# Set paper trading and data url's:
paper_url='https://paper-api.alpaca.markets'
data_url = 'wss://data.alpaca.markets'

In [ ]:
# Create trading API object:
alpaca_api=tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    paper_url,
    api_version='v2'
)

In [ ]:
# Retrieve and display account details:
account = alpaca_api.get_account()
print(account)

In [ ]:
# List Positions:
positions=alpaca_api.list_positions()
print(positions)

In [ ]:
# Create streaming API object:
stream = tradeapi.stream2.StreamConn(
    alpaca_api_key,
    alpaca_secret_key,
    base_url=paper_url,
    data_url=data_url,
    data_stream='alpacadatav1'
)

In [ ]:
# Put positions info into a DataFrame:
positions_df=pd.DataFrame({
    'asset_id':[x.asset_id for x in positions],
    'ticker':[x.symbol for x in positions],
    'cost_basis':[x.cost_basis for x in positions],
    'asset_id':[x.asset_id for x in positions],
    'current_price':[x.asset_id for x in positions],
    'lastday_price':[x.asset_id for x in positions],
    'change_today':[x.change_today for x in positions],
    'qty':[x.qty for x in positions],
    'market_value':[x.market_value for x in positions],
    'unrealized_intraday_pl':[x.unrealized_intraday_pl for x in positions],
    'unrealized_intraday_plpc':[x.unrealized_intraday_plpc for x in positions],
    'unrealized_pl':[x.unrealized_intraday_plpc for x in positions],
    'unrealized_plpc':[x.unrealized_intraday_plpc for x in positions]
})

# Export to .csv file:
positions_df.to_csv('Data/stock_positions.csv')

# Create tickers list from positions_df DataFrame:
positions_tickers=positions_df.ticker.tolist()

In [ ]:
# Function to stream account updates:
@stream.on(r'^account_updates$')
async def on_account_updates(stream, channel, account):
    print('account', account)
    
# Function to stream trade updates:
@stream.on(r'^trade_updates$')
async def on_trade_updates(stream, channel, trade):
    print('trade', trade)

# Check for message indicating trade has closed:
@stream.on(r'^trade_updates$')
async def on_trade_updates(stream, channel, trade):
	if trade.order['order_type'] != 'market' and trade.order['filled_qty'] == '100':
		# Trade closed - begin new trade:
		trade_params = set_trade_params(candlesticks.df.'IPDN')    

# Keep  data up to date so we don’t have to query the API for historical data next for the next order:
@stream.on(r'^AM.IPDN$')
async def on_minute_bars(stream, channel, bar):
    # Append the latest bar to candlesticks.df:
	if isinstance(candlesticks.df, pd.DataFrame):
		ts = pd.to_datetime(bar.timestamp, unit='ms')
		candlesticks.df.loc[ts] = [bar.open, bar.high, bar.low, bar.close, bar.volume]
    # If current high took out 10-bar high send a buy order:
	if not trade_params['trade_taken']:
		if bar.high > trade_params['high']:
			trade_params['trade_taken'] = send_order('buy', bar)
    # If current high took out 10-bar low send a sell order:
		elif bar.low < trade_params['low']:
			trade_params['trade_taken'] = send_order('sell', bar)
    # Check there's at least 900 seconds (15 min) left until market close or sleep script until market reopens:
	if time_to_market_close() > 900:
		wait_for_market_open()
        
        
# Function to start data stream:
def start_stream():
	stream.run(['account_updates', 'trade_updates'])

# Start the WebSocket thread:
ws_thread = threading.Thread(target=start_stream,daemon=True)
ws_thread.start()

In [ ]:
# Function to check if market is open and sleep until open if market is closed:
def time_to_market_open():
	clock = api.get_clock()
	if not clock.is_open:
		time_to_open = (clock.next_open - clock.timestamp).total_seconds()
		sleep(round(time_to_open))

In [ ]:
# Function to check time until market close:
def time_to_market_close():
	clock = api.get_clock()
	return (clock.next_close - clock.timestamp).total_seconds()

In [ ]:
# Function containing our trade parameters:
def set_trade_params(df):
	return {
		'high': df.high.tail(10).max(),
		'low': df.low.tail(10).min(),
		'trade_taken': False,
	}

In [ ]:
# Function to send an order once a target entry is identified:
def send_order(direction, bar):
    # No trades in last 15 mins of trading day so we can avoid mkt close volatility:
	if time_to_market_close() > 900:
		print(f'sent {direction} trade')
		range_size = trade_params['high'] - trade_params['low']

		if direction == 'buy':
			sl = bar.high - range_size
			tp = bar.high + range_size
		elif direction == 'sell':
			sl = bar.low + range_size
			tp = bar.low - range_size
        # Submit the order
			api.submit_order(
			symbol='IPDN',
			qty=100,
			side=direction,
			type='market',
			time_in_force='day',
			order_class='bracket',
			stop_loss=dict(stop_price=str(sl)),
			take_profit=dict(limit_price=str(tp)),
		)

		return True
    
    # Sleep:
	wait_for_market_open()
	return False

In [ ]:
# # Keep  data up to date so we don’t have to query the API for historical data next for the next order:
# @stream.on(r'^AM.IPDN$')
# async def on_minute_bars(stream, channel, bar):
#     # Append the latest bar to candlesticks.df:
# 	if isinstance(candlesticks.df, pd.DataFrame):
# 		ts = pd.to_datetime(bar.timestamp, unit='ms')
# 		candlesticks.df.loc[ts] = [bar.open, bar.high, bar.low, bar.close, bar.volume]
#     # If current high took out 10-bar high send a buy order:
# 	if not trade_params['trade_taken']:
# 		if bar.high > trade_params['high']:
# 			trade_params['trade_taken'] = send_order('buy', bar)
#     # If current high took out 10-bar low send a sell order:
# 		elif bar.low < trade_params['low']:
# 			trade_params['trade_taken'] = send_order('sell', bar)
#     # Check there's at least 900 seconds (15 min) left until market close or sleep script until market reopens:
# 	if time_to_market_close() > 900:
# 		wait_for_market_open()

In [ ]:
# # Check for message indicating trade has closed:
# @stream.on(r'^trade_updates$')
# async def on_trade_updates(stream, channel, trade):
# 	if trade.order['order_type'] != 'market' and trade.order['filled_qty'] == '100':
# 		# Trade closed - begin new trade:
# 		trade_params = set_trade_params(candlesticks.df.IPDN)

In [ ]:
candlesticks = api.get_barset('IPDN', 'minute', limit=10)
trade_params = set_trade_params(candlesticks.df.'IPDN')
stream.run(['AM.IPDN', 'trade_updates'])